# Search

In this homework, you'll implement a basic search engine by defining your own Python classes. A **search engine** is an algorithm that takes a query and retrieves the most relevant documents for that query. In order to identify the most relevant documents, our search engine will use **term frequency–inverse document frequency** ([tf–idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)), an information statistic for determining the relevance of a term to each document from a corpus consisting of many documents.

The **tf–idf statistic** is a product of two values: term frequency and inverse document frequency. **Term frequency** computes the number of times that a term appears in a **document** (such as a single Wikipedia page). If we were to use only the term frequency in determining the relevance of a term to each document, then our search result might not be helpful since most documents contain many common words such as "the" or "a". In order to downweight these common terms, the **document frequency** computes the number of times that a term appears across the **corpus** of all documents.

You will need the `doggos` and `small_wiki` directories to be copied to your work area in the same folder. You do not need to submit these folders but should submit this notebook file after `Run All` and saved.

In [24]:
# You must pass all mypy strict syntax checks

%pip install -q nb-mypy pytest ipytest
%reload_ext nb_mypy
%nb_mypy mypy-options --strict

Note: you may need to restart the kernel to use updated packages.


Version 1.0.6


In [25]:
%pip install -q pytest ipytest

import os
import math
import re

import pytest
import ipytest
ipytest.autoconfig()


def clean(token: str, pattern: re.Pattern[str] = re.compile(r"\W+")) -> str:
    """
    Returns all the characters in the token lowercased and without matches to the given pattern.

    >>> clean("Hello!")
    'hello'
    """
    return pattern.sub("", token.lower())

Note: you may need to restart the kernel to use updated packages.


## Outside Sources

Update the following Markdown cell to include your name and list your outside sources. Submitted work should be consistent with the curriculum and your sources.

**Name**: Abhiram Somisetty

1. Stack Overflow
2. pytest documentation
3. W3Schools


## Task: `Document`

Write and test a `Document` class in the code cell below that can be used to represent the text in a web page and includes methods to compute term frequency. (But not document frequency since that would require access to all the documents in the corpus.)

The `Document` class should include:

1. An initializer that takes a `str` path and initializes the document data based on the text in the specified file. Assume that the file exists, but that it could be empty. In order to implement `term_frequency` later, we'll need to precompute and save the term frequency for each term in the document in the initializer as a field by constructing a dictionary that maps each `str` term to its `float` term frequency. Term frequency is defined as *the count of the given term* divided by *the total number of words in the text*.
> Consider the term frequencies for this document containing 4 total words: "the cutest cutest dog".
>
> - "the" appears 1 time out of 4 total words, so its term frequency is 0.25.
> - "cutest" appears 2 times out of 4 total words, so its term frequency is 0.5.
> - "dog" appears 1 time out of 4 total words, so its term frequency is 0.25.

   When constructing this dictionary, call the `clean` function to convert the input token to lowercase and ignore non-letter characters so that "corgi", "CoRgi", and "corgi!!" are all considered the same string "corgi" to the search algorithm.

1. A method `term_frequency` that takes a given `str` term and returns its term frequency by looking it up in the precomputed dictionary. Remember to normalize the term before looking it up to find the corresponding match. If the term does not occur, return 0.

1. A method `get_path` that returns the `str` path of the file that this document represents.

1. A method `get_words` that returns a `set` of the unique, cleaned words in this document.

1. A method `__repr__` that returns a string representation of this document in the format `Document('{path}')` (with literal single quotes in the output) where `{path}` is the path to the document from the initializer. The `__repr__` method is called when Jupyter Notebook needs to display a `Document` as output, so we should be able to copy the string contents into a new code cell and immediately run it to create a new `Document`.

**For each of the 4 methods (excluding the initializer) in the `Document` class, write a testing function that contains at least 3 `pytest`-style assertions based on your own testing corpus and written as expressions like 1 / 5 that show your thinking**. As always, your test cases should expand the domain and range. Documentation strings are optional for testing functions.

- We've provided some example corpuses in the `doggos` directory and the `small_wiki` directory. For this task, create your own testing corpus by creating a **New Folder** and adding your own text files to it.
- Be sure to exhaustively test your `Document` class before moving on: bugs in `Document` will make implementing the following `SearchEngine` task much more difficult.
- Line numbers reported in error messages will be off by one due to the first line of code required to run `%%ipytest`. Look below the indicated line. Line comments (lines that begin `#`) may also result in different line numbering.

In [27]:
%%ipytest -v

class Document:
    def __init__(self, path: str) -> None:
        self._path = path
        self._term_frequencies: dict[str, float] = {}

        with open(path, 'r', encoding='utf-8') as f:
            content = f.read()
            raw_tokens = content.split()

        if not raw_tokens:
            return

        cleaned_words = [clean(token) for token in raw_tokens]
        cleaned_words = [w for w in cleaned_words if w]

        total_words = len(cleaned_words)
        word_counts: dict[str, int] = {}
        for word in cleaned_words:
            word_counts[word] = word_counts.get(word, 0) + 1

        for word, count in word_counts.items():
            self._term_frequencies[word] = count / total_words

    def term_frequency(self, word: str) -> float:
        return self._term_frequencies.get(clean(word), 0.0)

    def get_path(self) -> str:
        return self._path

    def get_words(self) -> set[str]:
        return set(self._term_frequencies.keys())

    def __repr__(self) -> str:
        return f"Document('{self._path}')"
class TestDocument:
    doc1 = Document("doggos/doc1.txt")
    doc2 = Document("doggos/doc2.txt")
    doc3 = Document("doggos/doc3.txt")
    sheep = Document("test/black_sheep.txt")
    humpty = Document("test/humpty_dumpty.txt")
    star = Document("test/twinkle_star.txt")
    euro = Document("small_wiki/Euro - Wikipedia.html")
    

    def test_term_frequency(self) -> None:
        assert self.doc1.term_frequency("dogs") == pytest.approx(1 / 5)
        assert self.euro.term_frequency("Euro") == pytest.approx(0.008651740942156932)
        assert self.doc1.term_frequency("missing") == pytest.approx(0.0)

        assert self.star.term_frequency("twinkle") == pytest.approx(5 / 85)
        assert self.humpty.term_frequency("dumpty") == pytest.approx(2 / 26)
        assert self.humpty.term_frequency("missing") == 0.0
        

        

    def test_get_path(self) -> None:
        assert self.doc2.get_path() == "doggos/doc2.txt"
        assert self.humpty.get_path() == "test/humpty_dumpty.txt"
        assert self.euro.get_path() == "small_wiki/Euro - Wikipedia.html"

        assert self.star.get_path() == "test/twinkle_star.txt"
        assert self.doc1.get_path() == "doggos/doc1.txt"
        assert self.sheep.get_path() == "test/black_sheep.txt"
        assert self.doc3.get_path() == "doggos/doc3.txt"
        assert self.humpty.get_path() == "test/humpty_dumpty.txt"

    def test_get_words(self) -> None:
        assert self.doc1.get_words() == set("dogs are the greatest pets".split())
        assert set(w for w in self.euro.get_words() if len(w) == 1) == set([
            *"0123456789acefghijklmnopqrstuvxyz".lower() # All one-letter words in Euro
        ])

        assert "wool" in self.sheep.get_words()
        assert "master" in self.sheep.get_words()
        assert len(self.sheep.get_words()) > 5

    def test_repr(self) -> None:
        assert repr(self.star) == "Document('test/twinkle_star.txt')"
        assert repr(self.doc2) == "Document('doggos/doc2.txt')"
        assert repr(self.euro) == "Document('small_wiki/Euro - Wikipedia.html')"
        assert repr(self.sheep) == "Document('test/black_sheep.txt')"
        assert "twinkle_star.txt" in repr(self.star)
        assert "doc2.txt" in repr(self.doc2)
        assert "test" in repr(self.humpty)

    

======================================= test session starts ========================================
platform linux -- Python 3.12.1, pytest-9.0.1, pluggy-1.6.0
rootdir: /workspaces/cse163-nchs/homework
plugins: anyio-4.11.0
collected 4 items

t_b5a82f699cf444b393ff94bfe3a92d0d.py ....                                                   [100%]

======================================== 4 passed in 0.02s =========================================


## Task: `SearchEngine`

Write and test a `SearchEngine` class in the code cell below that represents a corpus of `Document` objects and includes methods to compute the tf–idf statistic between a given query and every document in the corpus. The `SearchEngine` begins by constructing an **inverted index** that associates each term in the corpus to the list of `Document` objects that contain the term.

To iterate over all the files in a directory, call `os.listdir` to list all the file names and join the directory to the filename with `os.path.join`. The example below will print only the `.txt` files in the `doggos` directory.

In [28]:
path = "doggos"
extension = ".txt"
for filename in os.listdir(path):
    if filename.endswith(extension):
        print(os.path.join(path, filename))

doggos/doc1.txt
doggos/doc3.txt
doggos/doc2.txt


The `SearchEngine` class should include:

1. An initializer that takes a `str` path to a directory such as `"small_wiki"` and a `str` file extension and constructs an inverted index from the files in the specified directory matching the given extension. By default, the extension should be `".txt"`. Assume the string represents a valid directory, and that the directory contains only valid files. Do not recreate any behavior that is already done in the `Document` class—call the `get_words()` method! Create at most one `Document` per file.

1. A method `_calculate_idf` that takes a `str` term and returns the inverse document frequency of that term. If the term is not in the corpus, return 0. Inverse document frequency is defined by calling `math.log` on *the total number of documents in the corpus* divided by *the number of documents containing the given term*.

1. A method `__repr__` that returns a string representation of this search engine in the format `SearchEngine('{path}')` (with literal single quotes in the output) where `{path}` is the path to the directory specified in the initializer.

1. A method `search` that takes a `str` **query** consisting of one or more terms and returns a `list` of relevant document paths that match at least one of the cleaned terms sorted by descending tf–idf statistic: the product of the term frequency and inverse document frequency. If there are no matching documents, return an empty list.

**For each of the 3 methods (excluding the initializer) in the `SearchEngine` class, write a testing function that contains at least 3 `pytest`-style assertions based on your own testing corpus and written as expressions like 1 / 5 that show your thinking**. Test cases for `SearchEngine.__repr__` may use the given corpuses `doggos` and `small_wiki` in addition to your corpus. Documentation strings are optional for testing functions.

- The type checker can be too picky about the `sorted` `key` parameter: a type error here can be safely ignored if the code works as intended.

In [29]:
%%ipytest -v

class SearchEngine:
    def __init__(self, directory: str, extension: str = ".txt") -> None:
        self._directory = directory
        self._documents: list[Document] = []
        self._inverted_index: dict[str, list[Document]] = {}

        for filename in os.listdir(directory):
            if filename.endswith(extension):
                path = os.path.join(directory, filename)
                doc = Document(path)
                self._documents.append(doc)
                for word in doc.get_words():
                    if word not in self._inverted_index:
                        self._inverted_index[word] = []
                    self._inverted_index[word].append(doc)


    def _calculate_idf(self, term: str) -> float:
        cleaned = clean(term)
        if cleaned not in self._inverted_index:
            return 0.0
        return math.log(len(self._documents) / len(self._inverted_index[cleaned]))


    def __repr__(self) -> str:
        return f"SearchEngine('{self._directory}')"

    def search(self, query: str) -> list[str]:
        query_terms = [clean(t) for t in query.split() if clean(t)]
        if not query_terms:
            return []

        doc_scores: dict[Document, float] = {}
        for term in query_terms:
            idf = self._calculate_idf(term)
            for doc in self._inverted_index.get(term, []):
                tf = doc.term_frequency(term)
                score = tf * idf
                doc_scores[doc] = doc_scores.get(doc, 0.0) + score

        # Sort documents by score in descending order
        sorted_docs = sorted(doc_scores.items(), key=lambda item: item[1], reverse=True)
        return [doc.get_path() for doc, _ in sorted_docs]

class TestSearchEngine:
# Let mypy know these exist on instances:
    _doggos: SearchEngine
    _small_wiki: SearchEngine
    _engine: SearchEngine
    @classmethod
    def setup_class(cls) -> None:
        cls._doggos = SearchEngine("doggos")
        cls._small_wiki = SearchEngine("small_wiki", ".html")
        cls._engine = SearchEngine("test")

    @staticmethod
    def os_path(path: str, filename: str) -> str:
        return os.path.join(path, filename)

    def test_search(self) -> None:
        assert self._engine.search("little star") == [
            self.os_path("test", "twinkle_star.txt"),
            self.os_path("test", "black_sheep.txt"),
        ]
        assert self._engine.search("humpty dumpty") == [
            self.os_path("test", "humpty_dumpty.txt"),
        ]
        assert self._engine.search("black sheep") == [
            self.os_path("test", "black_sheep.txt"),
        ]
        assert self._engine.search("missing term") == []
        assert self._doggos.search("love dogs") == [
            self.os_path("doggos","doc3.txt"),
            self.os_path("doggos", "doc1.txt")]
        assert self._small_wiki.search("data")[:10] == [
            self.os_path("small_wiki", "Internet privacy - Wikipedia.html"),
            self.os_path("small_wiki", "Machine learning - Wikipedia.html"),
            self.os_path("small_wiki", "Bloomberg L.P. - Wikipedia.html"),
            self.os_path("small_wiki", "Waze - Wikipedia.html"),
            self.os_path("small_wiki", "Digital object identifier - Wikipedia.html"),
            self.os_path("small_wiki", "Chief financial officer - Wikipedia.html"),
            self.os_path("small_wiki", "UNCF - Wikipedia.html"),
            self.os_path("small_wiki", "Jackson 5 Christmas Album - Wikipedia.html"),
            self.os_path("small_wiki", "KING-FM - Wikipedia.html"),
            self.os_path("small_wiki", "The News-Times - Wikipedia.html"),
        ]
    def test_calculate_idf(self) -> None:
        assert self._doggos._calculate_idf("dogs") == pytest.approx(math.log(3 / 2))
        assert self._doggos._calculate_idf("missing") == 0.0
        assert self._doggos._calculate_idf("are") == pytest.approx(math.log(3 / 1))

        assert self._engine._calculate_idf("the") == 0.0
        assert self._engine._calculate_idf("little") == pytest.approx(math.log(3 / 2))
        assert self._engine._calculate_idf("humpty") == pytest.approx(math.log(3 / 1))
    def test_repr(self) -> None:
        assert repr(self._engine) == "SearchEngine('test')"
        assert "SearchEngine" in repr(self._engine)
        assert repr(self._doggos) == "SearchEngine('doggos')"
        assert "doggos" in repr(self._doggos)
        assert repr(self._small_wiki) == "SearchEngine('small_wiki')"
        assert "small_wiki" in repr(self._small_wiki)

======================================= test session starts ========================================
platform linux -- Python 3.12.1, pytest-9.0.1, pluggy-1.6.0
rootdir: /workspaces/cse163-nchs/homework
plugins: anyio-4.11.0
collected 3 items

t_b5a82f699cf444b393ff94bfe3a92d0d.py ...                                                    [100%]

======================================== 3 passed in 2.55s =========================================


We recommend the following iterative software development approach to implement the `search` method.

1. Write code to handle queries that contain only a single term by collecting all the documents that contain the given term, computing the tf–idf statistic for each document, and returning the list of document paths sorted by descending tf–idf statistic.

1. Write tests to ensure that your program works on single-term queries.

1. Write code to handle queries that contain more than one term by returning all the documents that match any of the terms in the query sorted by descending tf–idf statistic. The tf–idf statistic for a document that matches more than one term is defined as the sum of its constituent single-term tf–idf statistics.

1. Write tests to ensure that your program works on multi-term queries.

Here's a walkthrough of the `search` function from beginning to end. Say we have a corpus in a directory called `"doggos"` containing 3 documents with the following contents:

- `doggos/doc1.txt` with the text `Dogs are the greatest pets.`
- `doggos/doc2.txt` with the text `Cats seem pretty okay`
- `doggos/doc3.txt` with the text `I love dogs!`

The initializer should construct the following inverted index.

```python
{"dogs":     [doc1, doc3],
 "are":      [doc1],
 "the":      [doc1],
 "greatest": [doc1],
 "pets":     [doc1],
 "cats":     [doc2],
 "seem":     [doc2],
 "pretty":   [doc2],
 "okay":     [doc2],
 "i":        [doc3],
 "love":     [doc3]}
```

Searching this corpus for the multi-term query `"love dogs"` should return a list `["doggos/doc3.txt", "doggos/doc1.txt"]` by:

1. Finding all documents that match at least one query term. The word `"love"` is found in `doc3` while the word `"dogs"` is found in `doc1` and `doc3`.

1. Computing the tf–idf statistic for each matching document. For each matching document, the tf–idf statistic for a multi-word query `"love dogs"` is the sum of the tf–idf statistics for `"love"` and `"dogs"` individually.

   1. For `doc1`, the sum of 0 + 0.081 = 0.081. The tf–idf statistic for `"love"` is 0 because the term is not in `doc1`.

   1. For `doc3`, the sum of 0.366 + 0.135 = 0.501.

1. Returning the matching document paths sorted by descending tf–idf statistic.

After completing your `SearchEngine`, run the following cell to search our small Wikipedia corpus for the query "data". Try some other search queries too!

In [23]:
SearchEngine("small_wiki", ".html").search("data")

['small_wiki/Internet privacy - Wikipedia.html',
 'small_wiki/Machine learning - Wikipedia.html',
 'small_wiki/Bloomberg L.P. - Wikipedia.html',
 'small_wiki/Waze - Wikipedia.html',
 'small_wiki/Digital object identifier - Wikipedia.html',
 'small_wiki/Chief financial officer - Wikipedia.html',
 'small_wiki/UNCF - Wikipedia.html',
 'small_wiki/Jackson 5 Christmas Album - Wikipedia.html',
 'small_wiki/KING-FM - Wikipedia.html',
 'small_wiki/The News-Times - Wikipedia.html',
 'small_wiki/Robert Mercer (businessman) - Wikipedia.html',
 'small_wiki/Federal Bureau of Investigation - Wikipedia.html',
 'small_wiki/Viacom - Wikipedia.html',
 'small_wiki/Seattle Municipal Street Railway - Wikipedia.html',
 'small_wiki/Mandalay Bay - Wikipedia.html',
 'small_wiki/Hal Abelson - Wikipedia.html',
 'small_wiki/File_Googleplex-Patio-Aug-2014.JPG - Wikipedia.html',
 'small_wiki/IEEE Computer Society - Wikipedia.html',
 'small_wiki/Nintendo - Wikipedia.html',
 'small_wiki/Mountain View, California - Wi